# H4221 - Rapport Machine Learning

[a compléter]

Auteurs: 
- DUBILLOT Elise
- FLANDRE Corentin
- THOMAS Colin

## Introduction

L'objectif du projet est de tester différents modèles de [a compléter]

In [13]:
print("hello-world")

hello-world


## Chargement du jeu de données

In [14]:
import pandas as pd

df = pd.read_csv("kc_house_data.csv")
print(df)


               id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7   
1             2570      7242   

### Traitement du jeu de données

[blabla à compléter]

Un traitement sur la date est à réaliser. Nous faisons le choix d'identifier uniquement le mois de la vente pour l'utilisr plus tard. Le jeu de données prend en compte des dates entre Mai 2014 et Mai 2015

In [18]:
# print(df["date"])
print(f"size: {len(df)}")
print(df["date"])
for i in range(len(df)):
    year = int(df["date"][i][0:4])
    month = int(df["date"][i][4:6])
    # day = int(df["date"][i][6:8]) # donnee non utilise pour notre traitement
    new_date = "error"
    if year==2014:
        if month==5:
            new_date = "month1"
        elif month==6:
            new_date = "month2"    
        elif month==7:
            new_date = "month3" 
        elif month==8:
            new_date = "month4" 
        elif month==9:
            new_date = "month5" 
        elif month==10:
            new_date = "month6" 
        elif month==11:
            new_date = "month7"
        elif month==12:
            new_date = "month8"
    elif year==2015:
        if month==1:
            new_date = "month9"
        elif month==2:
            new_date = "month10"    
        elif month==3:
            new_date = "month11" 
        elif month==4:
            new_date = "month12" 
        elif month==5:
            new_date = "month13" 
    df["date"][i] = new_date
print("--------------------")
# print(df["date"])
print(df)
    
    


size: 21613
0         month6
1         month8
2        month10
3         month8
4        month10
          ...   
21608     month1
21609    month10
21610     month2
21611     month9
21612     month6
Name: date, Length: 21613, dtype: object


ValueError: invalid literal for int() with base 10: 'mont'

## Exploration du jeu de données

[blabla à compléter]